In [2]:
#load packages
library(ggplot2)
library(microbiome)
library(phyloseq)
library("tidyverse")
library("dplyr")
library("randomForestSRC")
library("ggpmisc")
library(patchwork)
library("speedyseq")
library(microViz)

In [3]:
packageVersion('randomForestSRC')

[1] ‘3.2.3’

In [4]:

load("/Users/stephanie.rosales/Documents/omics_projects/POR_16S/CoralPersistence/data/POR_counts_4_analysis.RData")
load("/Users/stephanie.rosales/Documents/omics_projects/POR_16S/CoralPersistence/data/POR_taxo_4_analysis.RData")


In [5]:
# import taxonomy 
taxo_mat <- POR_taxo_4_analysis %>%
  rownames_to_column %>%
  filter(rowname != 'ASV_2') %>%
column_to_rownames("rowname")  %>%
mutate(family = ifelse(row.names(.) == "ASV_94", "ASV_94:Bacteroidia", family)) %>% # add labels to taxonomy
#mutate(family = ifelse(row.names(.) == "ASV_68", "ASV_68", family)) %>%
  mutate(domain = ifelse(row.names(.) == "ASV_5", "ASV_5:core", domain)) %>%
  mutate(phylum = ifelse(row.names(.) == "ASV_5", "ASV_5:core", phylum)) %>%
  mutate(class = ifelse(row.names(.) == "ASV_5", "ASV_5:core", class)) %>%
   mutate(order = ifelse(row.names(.) == "ASV_5", "ASV_5:core", order)) %>%
  mutate(family = ifelse(row.names(.) == "ASV_5", "ASV_5:core", family)) %>%
mutate(genus = ifelse(row.names(.) == "ASV_5", "ASV_5:core", genus)) %>%
mutate(species = ifelse(row.names(.) == "ASV_5", "ASV_5:core", species)) %>%
 mutate(domain = ifelse(row.names(.) == "ASV_6", "ASV_6:core", domain)) %>%
  mutate(phylum = ifelse(row.names(.) == "ASV_6", "ASV_6:core", phylum)) %>%
  mutate(class = ifelse(row.names(.) == "ASV_6", "ASV_6:core", class)) %>%
   mutate(order = ifelse(row.names(.) == "ASV_6", "ASV_6:core", order)) %>%
  mutate(family = ifelse(row.names(.) == "ASV_6", "ASV_6:core", family)) %>%
mutate(genus = ifelse(row.names(.) == "ASV_6", "ASV_6:core", genus)) %>%
mutate(species = ifelse(row.names(.) == "ASV_6", "ASV_6:core", species)) %>%
mutate(class = ifelse(row.names(.) == "ASV_7", "SAR324:core", class)) %>%
   mutate(order = ifelse(row.names(.) == "ASV_7", "SAR324:core", order)) %>%
  mutate(family = ifelse(row.names(.) == "ASV_7", "SAR324:core", family)) %>%
mutate(genus = ifelse(row.names(.) == "ASV_7", "SAR324:core", genus)) %>%
mutate(species = ifelse(row.names(.) == "ASV_7", "SAR324:core", species)) %>%
mutate(genus = ifelse(row.names(.) == "ASV_23", "Cyanobiaceae:core", genus)) %>%
mutate(species = ifelse(row.names(.) == "ASV_23", "Cyanobiaceae:core", species)) 

In [6]:
meta = read.csv ("/Users/stephanie.rosales/Documents/omics_projects/POR_16S/CoralPersistence/metadata/POR_combo_meta.csv",
row.names=1, header = T)%>%
  mutate(
    Month = factor(Month, levels = month.name)
  ) %>%
  arrange(Month) %>%
arrange(collection_date)

ASV <- otu_table(as.matrix(POR_counts_4_analysis), 
                 taxa_are_rows = T)
TAX <- tax_table(as.matrix(taxo_mat))
SAMP <- sample_data(meta)

ps <- phyloseq(ASV, TAX, SAMP)
ps

phyloseq-class experiment-level object
otu_table()   OTU Table:          [ 27533 taxa and 363 samples ]:
sample_data() Sample Data:        [ 363 samples by 58 sample variables ]:
tax_table()   Taxonomy Table:     [ 27533 taxa by 7 taxonomic ranks ]:
taxa are rows

In [7]:
euk_asvs <- c("ASV_10673","ASV_10883",	"ASV_11806","ASV_12333",	"ASV_12335",	"ASV_13587",	"ASV_14757",	"ASV_15238",	"ASV_15259",	"ASV_16364",	"ASV_16936",	"ASV_17406",	"ASV_18096",	"ASV_18375",	"ASV_18682",	"ASV_19529",	"ASV_19795",	"ASV_19953",	"ASV_2",	"ASV_20640",	"ASV_2100",	"ASV_22284",	"ASV_22692",	"ASV_23038",	"ASV_23144",	"ASV_24135",	"ASV_24243",	"ASV_27014",	"ASV_29665",	"ASV_30547",	"ASV_30626",	"ASV_30884",	"ASV_4161",	"ASV_4498",	"ASV_476",	"ASV_4797",	"ASV_4966",	"ASV_5018",	"ASV_5118",	"ASV_5154",	"ASV_6282",	"ASV_653",	"ASV_6843",	"ASV_7264",	"ASV_7427",	"ASV_8",	"ASV_9248",	"ASV_9692")
allTaxa = taxa_names(ps)
allTaxa <- allTaxa[!(allTaxa %in% euk_asvs)]
ps_fill_euk = prune_taxa(allTaxa, ps)
# new phyloseq object with just the taxa you kept.
ps_fill_euk

phyloseq-class experiment-level object
otu_table()   OTU Table:          [ 27486 taxa and 363 samples ]:
sample_data() Sample Data:        [ 363 samples by 58 sample variables ]:
tax_table()   Taxonomy Table:     [ 27486 taxa by 7 taxonomic ranks ]:
taxa are rows

In [8]:
extras=c("apal_cf_gCN2_cl6_fr5_AH_2019-09-25",
  "apal_ndr_gCN2_cl15_fr4_AH_2019-11-19",
  "apal_pr_gCN2_cl75_fr6_AH_2019-11-18",
  "apal_ndr_gCN4_cl32_frB_AH_2019-11-19",
  "apal_pr_gCN4_cl98_fr4_AH_2019-09-26",
  "apal_cf_gHS1_cl15_frB_AH_2019-11-19",
  "apal_pr_gHS1_cl106_fr7_AH_2019-11-18",
  "apal_pr_gHS1_cl108_fr3_AH_2019-11-18",
  "apal_ndr_gML2_cl18_fr6_AH_2019-11-19",
  "apal_ndr_gCN1_cl17_fr4_AH_2019-09-25",
  "apal_pr_gCN4_cl98_fr4_AH_2019-09-26",
  "apal_pr_gHS1_cl106_fr6_AH_2019-07-02",
  "apal_pr_gHS1_cl106_fr7_AH_2019-03-01",
  "apal_pr_gHS1_cl106_fr7_AH_2019-11-18",
  "apal_pr_gHS1_cl108_fr3_AH_2019-11-18",
  "apal_pr_gML2_cl58_frA_AH_2019-11-18",
        "apal_pr_gCN2_cl81_fr6_AH_2019-09-26",
        "apal_pr_gCN2_cl81_fr4_AH_2019-09-26",
        "apal_ndr_gCN4_cl38_frB_AH_2019-11-19",
        "apal_ndr_gCN4_cl38_fr5_AH_2019-11-19",
        "apal_ndr_gCN1_cl17_fr4_AH_2019-09-25")


ps_fill_samp = subset_samples(ps_fill_euk, !sample_ID %in% extras)
ps_fill_samp

phyloseq-class experiment-level object
otu_table()   OTU Table:          [ 27486 taxa and 346 samples ]:
sample_data() Sample Data:        [ 346 samples by 58 sample variables ]:
tax_table()   Taxonomy Table:     [ 27486 taxa by 7 taxonomic ranks ]:
taxa are rows

In [9]:
ps_merge = merge_samples2(ps_fill_samp, "geno_clus")
ps_merge

phyloseq-class experiment-level object
otu_table()   OTU Table:          [ 27486 taxa and 124 samples ]:
sample_data() Sample Data:        [ 124 samples by 58 sample variables ]:
tax_table()   Taxonomy Table:     [ 27486 taxa by 7 taxonomic ranks ]:
taxa are rows

In [10]:
ps_fill = filter_taxa(ps_merge, function(x) sum(x > 5) > (0.1*length(x)), TRUE)  %>% tax_fix()
ps_fill 
ps_fill

Row named: ASV_10
contains no non-unknown values, returning:
'ASV_10' for all replaced levels.
Consider editing this tax_table entry manually.

Row named: ASV_13
contains no non-unknown values, returning:
'ASV_13' for all replaced levels.
Consider editing this tax_table entry manually.

Row named: ASV_21
contains no non-unknown values, returning:
'ASV_21' for all replaced levels.
Consider editing this tax_table entry manually.

Row named: ASV_25
contains no non-unknown values, returning:
'ASV_25' for all replaced levels.
Consider editing this tax_table entry manually.

Row named: ASV_68
contains no non-unknown values, returning:
'ASV_68' for all replaced levels.
Consider editing this tax_table entry manually.

Row named: ASV_88
contains no non-unknown values, returning:
'ASV_88' for all replaced levels.
Consider editing this tax_table entry manually.

Row named: ASV_110
contains no non-unknown values, returning:
'ASV_110' for all replaced levels.
Consider editing this tax_table entry m

phyloseq-class experiment-level object
otu_table()   OTU Table:          [ 1129 taxa and 124 samples ]:
sample_data() Sample Data:        [ 124 samples by 58 sample variables ]:
tax_table()   Taxonomy Table:     [ 1129 taxa by 7 taxonomic ranks ]:
taxa are rows

phyloseq-class experiment-level object
otu_table()   OTU Table:          [ 1129 taxa and 124 samples ]:
sample_data() Sample Data:        [ 124 samples by 58 sample variables ]:
tax_table()   Taxonomy Table:     [ 1129 taxa by 7 taxonomic ranks ]:
taxa are rows

In [11]:
#set seed
set.seed(2)

In [12]:
predictors <- t(otu_table(ps_fill))

# Make one column for response variable 
response <- as.factor(sample_data(ps_fill)$Samp_trip)

# Combine them into data frame
rf.data <- data.frame(response, predictors)
head(rf.data, n=2)


trip.classify <- rfsrc(response~., data = rf.data, ntree = 400,
                     importance="permute", csv.num=TRUE)
trip.classify

tax_trip=cbind(as.data.frame(trip.classify$importance), as.data.frame(tax_table(ps_fill))) %>%
arrange(-all) #%>%
#head(n=10)

head(tax_trip)




,response,ASV_479,ASV_893,ASV_365,ASV_535,ASV_2096,ASV_772,ASV_1957,ASV_1450,ASV_2272,⋯,ASV_4592,ASV_4841,ASV_5161,ASV_5172,ASV_5315,ASV_5568,ASV_13462,ASV_13967,ASV_20040,ASV_20228
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ML2_78_cf_Trip_3,Trip_3,0,0,31,47,0,0,0,10,0,⋯,0,0,0,0,0,0,0,0,0,0
CN4_35_cf_Trip_3,Trip_3,0,10,0,133,0,9,22,216,0,⋯,0,51,4,0,0,0,0,0,0,0


                         Sample size: 124
           Frequency of class labels: 34, 33, 27, 30
                     Number of trees: 400
           Forest terminal node size: 1
       Average no. of terminal nodes: 20.57
No. of variables tried at each split: 34
              Total no. of variables: 1129
       Resampling used to grow trees: swor
    Resample size used to grow trees: 78
                            Analysis: RF-C
                              Family: class
                      Splitting rule: gini *random*
       Number of random split points: 10
                   (OOB) Brier score: 0.08599295
        (OOB) Normalized Brier score: 0.45862909
                           (OOB) AUC: 0.99144798
   (OOB) Requested performance error: 0.01612903, 0.02941176, 0.03030303, 0, 0

Confusion matrix:

          predicted
  observed Trip_2 Trip_3 Trip_4 Trip_5 class.error
    Trip_2     33      1      0      0      0.0294
    Trip_3      1     32      0      0      0.0303
    Trip_4  

,all,Trip_2,Trip_3,Trip_4,Trip_5,domain,phylum,class,order,family,genus,species
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
ASV_865,0.020325902,0.037975996,-0.008237218,0.16611722,0.043039462,Bacteria,Proteobacteria,Gammaproteobacteria,Alteromonadales,Colwelliaceae,Thalassotalea,Thalassotalea genus
ASV_412,0.013403673,0.039974733,0.043245393,0.01006771,0.047569932,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,NS4 marine group,NS4 marine group genus
ASV_1966,0.010785930,0.027982313,0.063838437,0.01006771,0.009060939,Bacteria,Proteobacteria,Alphaproteobacteria,Parvibaculales,PS1 clade,PS1 clade family,PS1 clade family
ASV_364,0.010554542,-0.015989893,0.076194263,0.02768620,0.027182818,Bacteria,Verrucomicrobiota,Verrucomicrobiae,Opitutales,Puniceicoccaceae,Coraliomargarita,Coraliomargarita genus
ASV_151,0.009536777,0.003997473,0.006177913,0.03020313,0.067957046,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas,Pseudomonas genus
ASV_1038,0.008336934,0.049968416,0.018533740,0.01761849,0.000000000,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,NS2b marine group,NS2b marine group genus


In [23]:
##Combine top important ASVs from each sampling trip

# Create a list to store the results
result_list <- list()

# Define the column names for sorting
sort_columns <- c("Trip_2", "Trip_3", "Trip_4", "Trip_5")

# Loop through the sorting columns
for (col in sort_columns) {
  result <- cbind(as.data.frame(trip.classify$importance), as.data.frame(tax_table(ps_fill))) %>%
    arrange(desc(get(col))) %>%
    head() %>%
    distinct()  # Remove duplicate rows
    
  # Assign the result to the list
  result_list[[paste0(col, "_5")]] <- result
}

# Combine the results into a single data frame
rf_trip_df <- do.call(rbind, result_list)


rf_trip_df <- rf_trip_df %>%
  rownames_to_column("ID") %>%
  separate("ID", into = c("Trip", "ASV"), sep = "\\.", remove = FALSE) %>%
  mutate(Taxa = paste(ASV, order, sep = ":")) %>%
  unite("Trip_top", Trip, na.rm = TRUE, remove = FALSE) %>%
column_to_rownames("ID")


df_trip_long=rf_trip_df  %>%
rownames_to_column() %>%
  pivot_longer(cols = c(Trip_2, Trip_3, Trip_4, Trip_5), names_to = "variable") %>%
  group_by(variable)

head(df_trip_long, n=2) 

#write.csv(df_trip_long, "/Users/stephanie.rosales/Documents/omics_projects/POR_16S/CoralPersistence/metadata/rf_trip_df.csv")

rowname,Trip_top,Trip,ASV,all,domain,phylum,class,order,family,genus,species,Taxa,variable,value
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Trip_2_5.ASV_1038,Trip_2_5,Trip_2_5,ASV_1038,0.008336934,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,NS2b marine group,NS2b marine group genus,ASV_1038:Flavobacteriales,Trip_2,0.04996842
Trip_2_5.ASV_1038,Trip_2_5,Trip_2_5,ASV_1038,0.008336934,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,NS2b marine group,NS2b marine group genus,ASV_1038:Flavobacteriales,Trip_3,0.01853374


In [14]:
predictors <- t(otu_table(ps_fill))
dim(predictors)
# Make one column for our outcome/response variable 

response <- as.factor(sample_data(ps_fill)$reef)

# Combine them into 1 data frame
rf.data <- data.frame(response, predictors)
head(rf.data, n=2)


reef.classify <- rfsrc(response~., data = rf.data, ntree = 400,
                     importance="permute", csv.num=TRUE)
reef.classify

tax_reef= cbind(as.data.frame(reef.classify$importance), as.data.frame(tax_table(ps_fill))) %>%
arrange(-all) #%>%
#head()

tax_reef %>%
arrange(-ndr) %>%
head()



[1]  124 1129

,response,ASV_479,ASV_893,ASV_365,ASV_535,ASV_2096,ASV_772,ASV_1957,ASV_1450,ASV_2272,⋯,ASV_4592,ASV_4841,ASV_5161,ASV_5172,ASV_5315,ASV_5568,ASV_13462,ASV_13967,ASV_20040,ASV_20228
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ML2_78_cf_Trip_3,cf,0,0,31,47,0,0,0,10,0,⋯,0,0,0,0,0,0,0,0,0,0
CN4_35_cf_Trip_3,cf,0,10,0,133,0,9,22,216,0,⋯,0,51,4,0,0,0,0,0,0,0


                         Sample size: 124
           Frequency of class labels: 31, 47, 46
                     Number of trees: 400
           Forest terminal node size: 1
       Average no. of terminal nodes: 19.6025
No. of variables tried at each split: 34
              Total no. of variables: 1129
       Resampling used to grow trees: swor
    Resample size used to grow trees: 78
                            Analysis: RF-C
                              Family: class
                      Splitting rule: gini *random*
       Number of random split points: 10
                   (OOB) Brier score: 0.13729064
        (OOB) Normalized Brier score: 0.61780786
                           (OOB) AUC: 0.90829628
   (OOB) Requested performance error: 0.26612903, 0.80645161, 0.04255319, 0.13043478

Confusion matrix:

          predicted
  observed cf ndr pr class.error
       cf   6  14 11      0.8065
       ndr  2  45  0      0.0426
       pr   1   5 40      0.1304

      (OOB) Misclassificatio

,all,cf,ndr,pr,domain,phylum,class,order,family,genus,species
,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
ASV_109,0.007961133,-0.002192163,0.03759326,0.020682579,Bacteria,Proteobacteria,Alphaproteobacteria,Rhodobacterales,Rhodobacteraceae,Rhodobacteraceae family,Rhodobacteraceae family
ASV_94,0.004260258,-0.010960814,0.03614736,0.001477327,Bacteria,Bacteroidota,Bacteroidia,Bacteroidia class,ASV_94:Bacteroidia,ASV_94:Bacteroidia family,ASV_94:Bacteroidia family
ASV_273,0.007580987,-0.010960814,0.03470147,0.026591887,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,Flavobacteriaceae family,Flavobacteriaceae family
ASV_316,0.004298609,-0.019729465,0.02602610,0.017727925,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,NS5 marine group,NS5 marine group genus
ASV_223,0.005696488,0.008768651,0.02313431,0.011818617,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,NS5 marine group,NS5 marine group genus
ASV_172,0.007094397,-0.006576488,0.02168842,0.033978523,Bacteria,Proteobacteria,Gammaproteobacteria,Cellvibrionales,Halieaceae,OM60(NOR5) clade,OM60(NOR5) clade genus


In [16]:
##Combine top important ASVs from each reef
set.seed(2)
# Create a list to store the results
result_list <- list()

# Define the column names for sorting
sort_columns <- c("pr", "ndr", "cf")

# Loop through the sorting columns
for (col in sort_columns) {
  result <- cbind(as.data.frame(reef.classify$importance), as.data.frame(tax_table(ps_fill))) %>%
    arrange(desc(get(col))) %>%
    head(n=5) %>%
    distinct()  # Remove duplicate rows
    
  # Assign the result to the list
  result_list[[paste0(col, "_5")]] <- result
}

# Combine the results into a single data frame
rf_reef_df <- do.call(rbind, result_list)

rf_reef_df <- rf_reef_df %>%
  rownames_to_column("ID") %>%
  separate("ID", into = c("reef", "ASV"), sep = "\\.", remove = FALSE) %>%
  mutate(Taxa = paste(order, ASV, sep = ":")) %>%
  unite("reef_top", reef, na.rm = TRUE, remove = FALSE) %>%
column_to_rownames("ID")


df_reef_long=rf_reef_df  %>%
rownames_to_column() %>%
  pivot_longer(cols = c(cf, ndr, pr), names_to = "variable") %>%
  group_by(variable)

head(df_reef_long, n=5)

#write.csv(df_reef_long, "/Users/stephanie.rosales/Documents/omics_projects/POR_16S/CoralPersistence/metadata/rf_reef_df.csv",
 #        )

rowname,reef_top,reef,ASV,all,domain,phylum,class,order,family,genus,species,Taxa,variable,value
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
pr_5.ASV_172,pr_5,pr_5,ASV_172,0.007094397,Bacteria,Proteobacteria,Gammaproteobacteria,Cellvibrionales,Halieaceae,OM60(NOR5) clade,OM60(NOR5) clade genus,Cellvibrionales:ASV_172,cf,-0.006576488
pr_5.ASV_172,pr_5,pr_5,ASV_172,0.007094397,Bacteria,Proteobacteria,Gammaproteobacteria,Cellvibrionales,Halieaceae,OM60(NOR5) clade,OM60(NOR5) clade genus,Cellvibrionales:ASV_172,ndr,0.021688419
pr_5.ASV_172,pr_5,pr_5,ASV_172,0.007094397,Bacteria,Proteobacteria,Gammaproteobacteria,Cellvibrionales,Halieaceae,OM60(NOR5) clade,OM60(NOR5) clade genus,Cellvibrionales:ASV_172,pr,0.033978523
pr_5.ASV_273,pr_5,pr_5,ASV_273,0.007580987,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,Flavobacteriaceae family,Flavobacteriaceae family,Flavobacteriales:ASV_273,cf,-0.010960814
pr_5.ASV_273,pr_5,pr_5,ASV_273,0.007580987,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,Flavobacteriaceae family,Flavobacteriaceae family,Flavobacteriales:ASV_273,ndr,0.034701470
